In [127]:
#import packages
from dash import Dash, html, dcc
import dash_bootstrap_components as dbc
from dash.dependencies import Input, Output, State
import plotly.express as px
import plotly.graph_objs as go
import pandas as pd
import numpy as np
import statsmodels.api as sm

In [ ]:
    X = df['Age']
    Y = df['Weight']
    X = sm.add_constant(X)  
    fitted = sm.OLS(Y, X).fit()
    predicted = fitted.predict(X)
    fig.add_trace(
        go.Scatter(
            x=df['Age'],
            y=predicted,
            mode='lines',
            name='Fitted Line',
            line=dict(color='red', width=2)
        )
    )

In [134]:
# read cats dataset
cats = pd.read_csv('cats_dataset.csv')
#rename columns
cats.rename(columns={'Age (Years)':'Age','Weight (kg)':'Weight'},inplace= True)
#cats.groupby(["Age","Breed"], as_index=False).count()
#px.bar(cats.groupby(["Age","Breed"],as_index=False).count()["Weight"], x= "Age", y ="Breed")


cats

,Breed,Age,Weight,Color,Gender
0,Russian Blue,19,7,Tortoiseshell,Female
1,Norwegian Forest,19,9,Tortoiseshell,Female
2,Chartreux,3,3,Brown,Female
3,Persian,13,6,Sable,Female
4,Ragdoll,10,8,Tabby,Male
...,...,...,...,...,...
995,British Shorthair,19,5,Gray,Female
996,British Shorthair,11,2,Bicolor,Female
997,Savannah,12,5,Bicolor,Female
998,American Shorthair,8,3,Tortoiseshell,Female


In [129]:
age_per_breed = (
    cats
    .groupby(['Age', 'Breed'], as_index=False)
    .size()  # Count the number of entries per Age and Breed
    .rename(columns={'size': 'NumberOfCats'})  # Rename the count column for clarity
)
age_per_breed

,Age,Breed,NumberOfCats
0,1,Abyssinian,1
1,1,American Shorthair,1
2,1,Bengal,1
3,1,Birman,2
4,1,British Shorthair,2
...,...,...,...
464,19,Siamese,1
465,19,Siberian,1
466,19,Sphynx,2
467,19,Tonkinese,3


In [135]:
app = Dash(__name__)
app.layout = html.Div([
    html.H1('Average weight per breed'),
    dcc.RadioItems(id='gender-selector',options=['Female','Male'],value='Female'),
    dcc.Graph(id="graph1"),    
])
@app.callback(Output('graph1','figure'),Input('gender-selector','value'))

def bar_chart_weight(gender):
    df = cats[cats["Gender"]==gender]
    fig = px.bar(df.groupby("Breed").mean(),y='Weight')
    return fig
if __name__ == "__main__":
    app.run(debug=True)

In [140]:
app = Dash(__name__)
app.layout = html.Div([
    html.H1('Select cat gender'),
    dcc.Checklist(id='gender-selector',options=['Female','Male'],value=['Female','Male']),
    html.Div(id='error-message', style={'color': 'red', 'fontSize': 20}),
    dcc.Graph(id="graph1"),
    
])
@app.callback(Output('graph1','figure'),Output('error-message', 'children'), Input('gender-selector','value'))

def bar_chart_weight(gender):
    if len(gender)==0:
        return {}, "Please select at least one gender."
    if len(gender)==2:
        fig = px.bar(cats.groupby("Breed").mean(),y='Weight',title = f"Average weight per breed (Male AND Female)")
        fig.update_yaxes(title = "Weight (in kgs)")
        return fig, ""
    else: 
        df = cats[cats["Gender"]==gender[0]]
        fig = px.bar(df.groupby("Breed").mean(),y='Weight', title = f"Average weight per breed {gender[0]}")
        return fig, ""
if __name__ == "__main__":
    app.run(debug=True)

In [143]:
age_per_breed = (
    cats
    .groupby(['Age', 'Breed'], as_index=False)
    .size()  # Count the number of entries per Age and Breed
    .rename(columns={'size': 'NumberOfCats'})  # Rename the count column for clarity
)

app = Dash(__name__)
app.layout = html.Div([
    dcc.Tabs(className = "dbc", children = [
        dbc.Tab(label="Tab 1", children=[
            dbc.Row([html.H1("Specify age range"),
                    dcc.RangeSlider(id='age',min = 0, max = cats["Age"].max(), step=1,value=[5,15]),
            ]),
            dbc.Row([
            dbc.Col([html.H1('Choose cat breed(s)'),
                    dcc.Dropdown(id="breed-selector",options = [{'label': breed, 'value': breed} for breed in cats['Breed'].unique()],value=["Sphynx","Ragdoll"],multi=True),
                    dcc.Graph(id="graph2")]),
            dbc.Col(dcc.Graph(id="graph3"))
            ])
        ]),
        dbc.Tab(label="Tab 2", children=[
            dbc.Row([html.H1('Select cat gender'),
            dcc.Checklist(id='gender-selector',options=['Female','Male'],value=['Female','Male']),
            html.Div(id='error-message', style={'color': 'red', 'fontSize': 20}),
            dcc.Graph(id="graph1")]),
            
        ])
    ])
    ,
])
@app.callback(Output('graph1','figure'),Output('error-message', 'children'), Input('gender-selector','value'))
def bar_chart_weight(gender):
    if len(gender)==0:
        return {}, "Please select at least one gender."
    if len(gender)==2:
        fig = px.bar(cats.groupby("Breed").mean(),y='Weight',title = f"Average weight per breed (Male AND Female)")
        fig.update_yaxes(title = "Weight (in kgs)")
        return fig, ""
    else: 
        df = cats[cats["Gender"]==gender[0]]
        fig = px.bar(df.groupby("Breed").mean(),y='Weight', title = f"Average weight per breed {gender[0]}")
        return fig, ""
    
@app.callback(Output('graph2','figure'), Output('graph3','figure'), Input('age','value'),Input('breed-selector','value'))
   
def age_bubble(age,breedselect):
    df = cats[cats['Age'].between(age[0], age[1]) & cats['Breed'].isin(list(breedselect))]
    fig = px.scatter(df, x="Age", y="Weight", color="Breed", title = f"Weight vs Age")
    unique_breeds = df['Breed'].unique()
    colors = px.colors.qualitative.Plotly
    extended_colors = colors * (len(unique_breeds) // len(colors) + 1)
    color_map = {breed: color for breed, color in zip(unique_breeds, extended_colors)}

    for breed in unique_breeds:
        breed_data = df[df['Breed'] == breed]
        X = breed_data['Age']
        Y = breed_data['Weight']
        X = sm.add_constant(X)  
        fitted = sm.OLS(Y, X).fit()
        predictions = fitted.predict(X)

        
        fig.add_trace(
            go.Scatter(
                x=breed_data['Age'],
                y=predictions,
                mode='lines',
                name=f'{breed} Fitted Line',
                line=dict(color=color_map[breed], width=2),
                showlegend=False
            )
        )
    fig.update_yaxes(title = "Weight (in kgs)")
    fig.update_xaxes(title = "Age (in years)")
    df2 = age_per_breed.query("@age[0] <= Age <= @age[1]")
    fig2 = px.bar(df2, x= "Age",y="NumberOfCats",color="Breed",barmode="group")
    fig2.update_yaxes(title = "Count of cats")
    fig2.update_xaxes(title = "Age (in years)")
    return fig, fig2

    
if __name__ == "__main__":
    app.run(debug=True)



In [ ]:
#import packages
from dash import Dash, html, dcc
import dash_bootstrap_components as dbc
from dash.dependencies import Input, Output, State
import plotly.express as px
import plotly.graph_objs as go
import pandas as pd
import numpy as np
import statsmodels.api as sm

# read cats dataset
cats = pd.read_csv('cats_dataset.csv')
#rename columns
cats.rename(columns={'Age (Years)':'Age','Weight (kg)':'Weight'},inplace= True)

In [146]:

age_per_breed = (
    cats
    .groupby(['Age', 'Breed'], as_index=False)
    .size()  # Count the number of entries per Age and Breed
    .rename(columns={'size': 'NumberOfCats'})  # Rename the count column for clarity
)

app = Dash(__name__)
app.layout = html.Div([
    dcc.Tabs(className = "dbc", children = [
        dbc.Tab(label="Tab 1", children=[
            dbc.Row([
                dbc.Col(html.H1("Specify age range"), width=12),
                dbc.Col(dcc.RangeSlider(id='age', min=0, max=cats["Age"].max(), step=1, value=[5, 15]), width=12)
            ]),
            dbc.Row([
            dbc.Col([html.H1('Choose cat breed(s)'),
                    dcc.Dropdown(id="breed-selector",options = [{'label': breed, 'value': breed} for breed in cats['Breed'].unique()],value=["Sphynx","Ragdoll"],multi=True),
                    dcc.Graph(id="graph2")], width=6),
            dbc.Col(dcc.Graph(id="graph3"), width=6)
            ])
        ]),
        dbc.Tab(label="Tab 2", children=[
            dbc.Row([html.H1('Select cat gender'),
            dcc.Checklist(id='gender-selector',options=['Female','Male'],value=['Female','Male']),
            html.Div(id='error-message', style={'color': 'red', 'fontSize': 20}),
            dcc.Graph(id="graph1")]),
            dbc.Row([dcc.Graph(id="color-bar-chart")]),
        ])
    ])
    ,
])
@app.callback(Output('graph1','figure'),Output('error-message', 'children'), Input('gender-selector','value'))
def bar_chart_weight(gender):
    if len(gender)==0:
        return {}, "Please select at least one gender."
    if len(gender)==2:
        fig = px.bar(cats.groupby("Breed").mean(),y='Weight',title = f"Average weight per breed (Male AND Female)")
        fig.update_yaxes(title = "Weight (in kgs)")
        return fig, ""
    else: 
        df = cats[cats["Gender"]==gender[0]]
        fig = px.bar(df.groupby("Breed").mean(),y='Weight', title = f"Average weight per breed {gender[0]}")
        return fig, ""
    
@app.callback(Output('graph2','figure'), Output('graph3','figure'), Input('age','value'),Input('breed-selector','value'))
   
def age_bubble(age,breedselect):
    df = cats[cats['Age'].between(age[0], age[1]) & cats['Breed'].isin(list(breedselect))]
    fig = px.scatter(df, x="Age", y="Weight", color="Breed", title = f"Weight vs Age")
    unique_breeds = df['Breed'].unique()
    colors = px.colors.qualitative.Plotly
    extended_colors = colors * (len(unique_breeds) // len(colors) + 1)
    color_map = {breed: color for breed, color in zip(unique_breeds, extended_colors)}

    for breed in unique_breeds:
        breed_data = df[df['Breed'] == breed]
        X = breed_data['Age']
        Y = breed_data['Weight']
        X = sm.add_constant(X)  
        fitted = sm.OLS(Y, X).fit()
        predictions = fitted.predict(X)

        
        fig.add_trace(
            go.Scatter(
                x=breed_data['Age'],
                y=predictions,
                mode='lines',
                name=f'{breed} Fitted Line',
                line=dict(color=color_map[breed], width=2),
                showlegend=False
            )
        )
    fig.update_yaxes(title = "Weight (in kgs)")
    fig.update_xaxes(title = "Age (in years)")
    df2 = age_per_breed.query("@age[0] <= Age <= @age[1]")
    fig2 = px.bar(df2, x= "Age",y="NumberOfCats",color="Breed",barmode="group")
    fig2.update_yaxes(title = "Count of cats")
    fig2.update_xaxes(title = "Age (in years)")
    return fig, fig2
@app.callback(
    Output('color-bar-chart', 'figure'),
    [Input('graph1', 'clickData')]
)
def update_color_chart(clickData):
    if clickData is None:
        return px.bar(title="Click on a bar to see color distribution")
    
    breed = clickData['points'][0]['x']
    df = cats[cats['Breed'] == breed].groupby('Color').size().reset_index(name='Count')
    
    fig = px.bar(df, x='Color', y='Count', title=f'Color Distribution for {breed}')
    fig.update_yaxes(title="Number of Cats")
    fig.update_xaxes(title="Color")
    return fig
    
if __name__ == "__main__":
    app.run(debug=True)

